In [ ]:
import os
import numpy as np
import pathlib
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/MyDrive/rice-leaf-disease/training"

dataset = []
label = 0

for file in os.listdir(data_dir):
    path = os.path.join(data_dir, file)
    for img in os.listdir(path):
        image = load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(32,32))
        image = img_to_array(image)
        image = image/255.0
        dataset += [[image, label]]
    label+=1

print('There are', len(dataset), 'images in', label, 'classes')

There are 16000 images in 4 classes


In [ ]:
data, labels = zip(*dataset)
data = np.array(data)
labels = np.array(to_categorical(labels))

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=44)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(12800, 32, 32, 3)
(3200, 32, 32, 3)
(12800, 4)
(3200, 4)


In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=20,
                                   zoom_range=0.1,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   fill_mode="nearest")

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1024, activation = 'relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
history = model.fit(train_datagen.flow(train_data, train_labels, batch_size=100),
                    validation_data=(val_data, val_labels),
                    epochs=30)

Epoch 1/30
128/128 [==============================] - 12s 90ms/step - loss: 0.9328 - accuracy: 0.5116 - val_loss: 0.7800 - val_accuracy: 0.6353
Epoch 2/30
128/128 [==============================] - 11s 88ms/step - loss: 0.7278 - accuracy: 0.6570 - val_loss: 0.4923 - val_accuracy: 0.7881
Epoch 3/30
128/128 [==============================] - 12s 90ms/step - loss: 0.4582 - accuracy: 0.8170 - val_loss: 0.3866 - val_accuracy: 0.8406
Epoch 4/30
128/128 [==============================] - 11s 88ms/step - loss: 0.3509 - accuracy: 0.8581 - val_loss: 0.2114 - val_accuracy: 0.9197
Epoch 5/30
128/128 [==============================] - 11s 88ms/step - loss: 0.2598 - accuracy: 0.8984 - val_loss: 0.1623 - val_accuracy: 0.9381
Epoch 6/30
128/128 [==============================] - 11s 89ms/step - loss: 0.2298 - accuracy: 0.9102 - val_loss: 0.1664 - val_accuracy: 0.9397
Epoch 7/30
128/128 [==============================] - 11s 88ms/step - loss: 0.2077 - accuracy: 0.9159 - val_loss: 0.2546 - val_accuracy:

In [ ]:
export_dir = '/content/drive/MyDrive/PadE/saved_model'
tf.saved_model.save(model,export_dir=export_dir)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/PadE/saved_model/assets


In [ ]:
export_dir = '/content/drive/MyDrive/PadE/model'
model.save(export_dir)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/PadE/model/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert();
tflite_model_file = pathlib.Path('/content/drive/MyDrive/PadE/saved_model/model.tflite')
tflite_model_file.write_bytes(tflite_model)

859168